In [207]:
import numpy as np
import csv
import pandas as pd
import matplotlib.pyplot as plt

In [208]:
def calc_sec(time):
    hms = time.split(':')
    hms = [float(x) for x in hms]
    sec = hms[2] + hms[1]*60 + hms[0]*3600
    sec = round(sec,3)
    return sec

In [209]:
def calc_ts(sec):
    ts = ''
    hr = int(sec/3600)
    mn = int((sec - (hr*3600))/60)
    sc = sec - (hr*3600) - (mn*60)
    sc = round(sc,3)
    ts += str(hr) + '-' + str(mn) + '-' + str(sc)
    # print(ts)
    return ts

In [210]:
def calc_t_period(dates,secs):
    t_period = []
    
    start_sec = secs[0]
    prev_sec = secs[0]
    prev_date = dates[0]

    for i in range(len(secs)):
        curr_sec = secs[i]
        diff_sec = curr_sec - prev_sec
        curr_date = dates[i]
        
        if((diff_sec>3.0) and (curr_date==prev_date)):
            t_period.append([curr_date,start_sec,prev_sec])
            start_sec = curr_sec
        elif(curr_date!=prev_date):
            t_period.append([prev_date,start_sec,prev_sec])
            start_sec = curr_sec
            prev_date = curr_date
        elif(i==len(secs)-1):
            t_period.append([curr_date,start_sec,curr_sec])

        prev_sec = curr_sec
    
    return t_period

## Partition accelerometer data

In [211]:
filepath = 'SmartwatchData/prep_data/data_acc_3.csv'

raw_df = pd.read_csv(filepath)

cols = raw_df.columns.tolist()

raw_df = raw_df[cols]

row_len = len(raw_df.index)

dates = [raw_df.iloc[i,0].split(' ')[0] for i in range(row_len)]
secs = [calc_sec(raw_df.iloc[i,0].split(' ')[1]) for i in range(row_len)]

raw_df['date'] = pd.Series(dates, index=raw_df.index)
raw_df['sec'] = pd.Series(secs, index=raw_df.index)

period = calc_t_period(dates,secs)

ts = []
for elem in period:
    ts.append([elem[0],calc_ts(elem[1]),calc_ts(elem[2])])
    
ts = np.array(ts)
print(ts)

[['2018-10-05' '14-42-28.999' '14-42-49.885']
 ['2018-10-05' '19-24-9.96' '19-24-32.733']
 ['2018-10-05' '19-50-43.588' '19-51-14.564']
 ['2018-10-05' '19-51-31.243' '19-51-36.394']
 ['2018-10-05' '19-52-31.353' '19-52-56.227']
 ['2018-10-19' '23-28-59.037' '23-29-54.682']
 ['2018-10-20' '11-21-19.914' '11-21-50.806']
 ['2018-10-20' '11-22-19.251' '11-22-38.652']
 ['2018-10-21' '17-39-29.338' '17-39-59.362']
 ['2018-10-21' '17-41-12.254' '17-41-21.864']
 ['2018-10-21' '17-41-33.649' '17-42-3.774']
 ['2018-10-21' '17-42-56.251' '17-43-0.27']
 ['2018-10-21' '17-43-43.9' '17-44-14.129']
 ['2018-10-21' '17-44-24.262' '17-44-28.909']
 ['2018-10-21' '17-45-56.587' '17-46-26.928']
 ['2018-10-21' '17-47-26.232' '17-47-33.278']
 ['2018-10-21' '17-47-45.516' '17-48-15.803']
 ['2018-10-21' '17-49-27.266' '17-49-36.082']
 ['2018-10-21' '18-45-53.452' '18-46-23.796']
 ['2018-10-21' '18-46-56.229' '18-47-3.126']
 ['2018-10-22' '16-4-47.86' '16-5-18.765']
 ['2018-10-22' '16-6-14.266' '16-6-25.488']
 

In [212]:
with open('SmartwatchData/timestamp_part.txt','w') as ts_label:
    for row in ts:
        line = (' ').join(row)
        ts_label.write(line + '\n')

In [213]:
count = 0
cols = ['timestamp','x','y','z']
filepath = 'SmartwatchData/partitioned_data/'

for elem in period:
    date = elem[0]
    start_t = elem[1]
    end_t = elem[2]
    
    if(end_t-start_t>10):
        for i in range(row_len):
            if(date==raw_df.iloc[i,-2] and start_t==raw_df.iloc[i,-1]):
                start_id = i
            if(date==raw_df.iloc[i,-2] and end_t==raw_df.iloc[i,-1]):
                end_id = i
                count += 1
                new_df = raw_df[start_id:end_id][cols]

                filename = 'acc_' + str(count) + '_' + calc_ts(start_t)

                new_df.to_csv(filepath + filename + '.csv')

In [214]:
from os import listdir, rename
from os.path import isfile, join

In [215]:
timelabel = []
with open('SmartwatchData/timestamp_label.txt','r') as file:
    for line in file:
        temp = []
        for elem in line.strip('\n').split(' '):
            elem = elem.replace(":","-")
            print(elem)
            temp.append(elem)
        timelabel.append(temp)
        
print(len(timelabel))
print(np.array(timelabel))

2018-10-05
14-42-28.999
14-42-49.885
walking_down
2018-10-05
19-24-9.96
19-24-32.733
walking_down
2018-10-05
19-50-43.588
19-51-14.564
walking
2018-10-05
19-51-31.243
19-51-36.394
walking
2018-10-05
19-52-31.353
19-52-56.227
walking_up
2018-10-19
23-28-59.037
23-29-54.682
nan
2018-10-20
11-21-19.914
11-21-50.806
walking
2018-10-20
11-22-19.251
11-22-38.652
nan
2018-10-21
17-39-29.338
17-39-59.362
walking
2018-10-21
17-41-12.254
17-41-21.864
walking
2018-10-21
17-41-33.649
17-42-3.774
walking_up
2018-10-21
17-42-56.251
17-43-0.27
nan
2018-10-21
17-43-43.9
17-44-14.129
walking_up
2018-10-21
17-44-24.262
17-44-28.909
nan
2018-10-21
17-45-56.587
17-46-26.928
standing
2018-10-21
17-47-26.232
17-47-33.278
nan
2018-10-21
17-47-45.516
17-48-15.803
standing
2018-10-21
17-49-27.266
17-49-36.082
nan
2018-10-21
18-45-53.452
18-46-23.796
running
2018-10-21
18-46-56.229
18-47-3.126
nan
2018-10-22
16-4-47.86
16-5-18.765
nan
2018-10-22
16-6-14.266
16-6-25.488
walking
2018-10-22
16-6-56.148
16-7-26.466

In [216]:
path = 'SmartwatchData/partitioned_data'
part_files = [f for f in listdir(path) if isfile(join(path, f))]
length = len(part_files) - 1

acc_str = 'acc_'
s1 = len(acc_str)

part_dict = {}

for e in part_files:
    cnt = 0
    idx = 0
    for ch in e:
        idx += 1
        if(ch=='_'):
            cnt += 1
        if(cnt==2):
            s2 = idx
            break
    s3 = len(e) - s2 + 1
    
    if(len(e)>17):
        ts = e[s2:-4]
        cut = e[s1:]
        num = cut[:-s3]
        
        part_dict[int(num)] = ts

In [219]:
print(len(part_dict))

27


## Rename partitioned data to labeled data

### "Can be run only once!"

In [218]:
num = 0
for i in range(len(timelabel)):
    row = timelabel[i]
    if(row[1] in part_dict.values() and row[3]!='nan'):
        for key, val in part_dict.items():
            if(val==row[1]):
                num = key
                
        old_name = 'SmartwatchData/partitioned_data/acc_' + str(num) + '_' + row[1] + '.csv'
        new_name = 'SmartwatchData/labeled_data/acc_' + row[3] + '_' + row[1] + '.csv'
        rename(old_name, new_name)